In [1]:
import pandas as pd
import math
from datetime import datetime, timedelta
import numpy as np
import numpy
from scipy.fftpack import fft, ifft
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
import scipy
from sklearn.decomposition import PCA
from collections import Counter
from apyori import apriori
# from mlxtend.frequent_patterns import association_rules

In [2]:
def readCSV(cgm, insulin):

    date_format_str = '%d/%m/%Y %H:%M:%S'
    CGMData = []

    CGMData =  pd.read_csv(cgm,low_memory=False)
    CGMDataClean = []

    for i in range(0,len(CGMData),1):

        arr = []
        month = int(str(CGMData['Date'][i]).split('/',3)[0])
        day = int(str(CGMData['Date'][i]).split('/',3)[1])
        year = int(str(CGMData['Date'][i]).split('/',3)[2])

        hour = int(str(CGMData['Time'][i]).split(':',3)[0])
        minute = int(str(CGMData['Time'][i]).split(':',3)[1])
        second = int(str(CGMData['Time'][i]).split(':',3)[2])
        date = str(day) + '/'+ str(month) +'/'+ str(year) + ' ' +str(hour)+':'+ str(minute)+':'+ str(second)
        dateAndTime = datetime.strptime(date, date_format_str)
        arr.append(dateAndTime)

        if CGMData['Sensor Glucose (mg/dL)'][i] != '' and CGMData['ISIG Value'][i] != '':

            arr.append(float(CGMData['Sensor Glucose (mg/dL)'][i]))
            arr.append(float(CGMData['ISIG Value'][i]))
            CGMDataClean.append(arr)

    CGMDataClean = CGMDataClean[::-1]
    InsulinData = []
    InsulinData =  pd.read_csv(insulin,low_memory=False)
    InsulinDataClean = []

    for i in range(0,len(InsulinData),1):

        row = []
        row2 = []
        date = []
        time = []

        month = int(str(InsulinData['Date'][i]).split('/',3)[0])
        day = int(str(InsulinData['Date'][i]).split('/',3)[1])
        year = int(str(InsulinData['Date'][i]).split('/',3)[2])
        hour = int(str(InsulinData['Time'][i]).split(':',3)[0])
        minute = int(str(InsulinData['Time'][i]).split(':',3)[1])
        second = int(str(InsulinData['Time'][i]).split(':',3)[2])

        date.append(year)
        date.append(month)
        date.append(day)

        time.append(hour)
        time.append(minute)
        time.append(second)

        row.append(date)
        row.append(time)
        row2.append(row)

    
        if InsulinData['BWZ Estimate (U)'][i] != '':

            if float(InsulinData['BWZ Estimate (U)'][i]) > 0 :

                row2.append(float(InsulinData['BWZ Estimate (U)'][i]))


                InsulinDataClean.append(row2)


    InsulinDataClean = InsulinDataClean[::-1]

    return InsulinDataClean, CGMDataClean


In [3]:
#X = np.array([[1, 2], [2, 2], [2, 3], [8, 7], [8, 8], [25, 80]])

In [4]:
def dataForFeatures(featureSpace, LabelRange):

    

    array = []
    for i in LabelRange:
       

        subarray = []
        lower = i[0]
        upper = i[1]

        subarrayTime = []
        subarrayGlucose = []
        subarrayISMG = []

        #repeats multiple times and is returning 0
        for j in featureSpace:

            glucose = math.isnan(j[1])
            #ismg = math.isnan(j[2])

            if j[0] >= lower and j[0] <= upper and glucose == False:

                #subarrayTime.append(j[0])
                subarrayGlucose.append(j[1])
                #subarrayISMG.append(j[2])

        #bin size
        if len(subarrayGlucose) > 0:

            #subarray.append(subarrayTime)
            #subarray.append(subarrayGlucose)
            #subarray.append(subarrayISMG)
           

            subarrayGlucose = subarrayGlucose
            array.append([subarrayGlucose,i[2]])

    return array


In [5]:
DataClean2, CGMDateAndFood2 = readCSV( 'CGMData.csv', 'InsulinData.csv')

In [6]:
def extraction(data):


    mealData = []



    i = 1
    


    while i < len(data) - 1:
            

            
        
        
            Year = data[i][0][0][0]

            Month = data[i][0][0][1]

            Day = data[i][0][0][2]


            Hour = data[i][0][1][0]

            Minute = data[i][0][1][1]

            Second = data[i][0][1][2]


            lowerYear = data[i-1][0][0][0]

            lowerMonth = data[i-1][0][0][1]

            lowerDay = data[i-1][0][0][2]


            lowerHour = data[i-1][0][1][0]

            lowerMinute = data[i-1][0][1][1]

            lowerSecond = data[i-1][0][1][2]


            upperYear = data[i+1][0][0][0]

            upperMonth = data[i+1][0][0][1]

            upperDay = data[i+1][0][0][2]


            upperHour = data[i+1][0][1][0]

            upperMinute = data[i+1][0][1][1]

            upperSecond = data[i+1][0][1][2]



            date_1 = str(lowerDay) + '/'+ str(lowerMonth) +'/'+ str(lowerYear) + ' ' +str(lowerHour)+':'+ str(lowerMinute)+':'+ str(lowerSecond)

            date_2 = str(upperDay) + '/'+ str(upperMonth) +'/'+ str(upperYear) + ' ' +str(upperHour)+':'+ str(upperMinute)+':'+ str(upperSecond)

            date_3 = str(Day) + '/'+ str(Month) +'/'+ str(Year) + ' ' +str(Hour)+':'+ str(Minute)+':'+ str(Second)

            date_format_str = '%d/%m/%Y %H:%M:%S'


            lower = datetime.strptime(date_1, date_format_str)

            upper =   datetime.strptime(date_2, date_format_str)

            current =   datetime.strptime(date_3, date_format_str)


            diffUpper = upper - current

            diffLower = current - lower

            diffUpper_in_hours = diffUpper.total_seconds() / 3600

            diffLower_in_hours = diffLower.total_seconds() / 3600


            if diffLower_in_hours > 0.5 and diffUpper_in_hours < 2:




                lowerTime = upper + timedelta(hours=0) - timedelta(hours=0.5)

                upperTime = upper + timedelta(hours=2)

                mealData.append([lowerTime,upperTime,data[i+1][1]])


                i = i + 1
            elif diffLower_in_hours > 0.5 and diffUpper_in_hours > 2.5:


                lowerTime = current + timedelta(hours=0) - timedelta(hours=0.5)

                upperTime = current + timedelta(hours=2)

                mealData.append([lowerTime,upperTime,data[i][1]])
                
               


            elif diffLower_in_hours < 0.5:

                mealData = mealData[:-1]
              


            i = i+1


    return mealData


In [7]:
meals = extraction(DataClean2)


In [8]:
mealpreFeatureData = dataForFeatures(CGMDateAndFood2, meals)



In [9]:
allMeal = []
for i in mealpreFeatureData:
    for j in i[0]:
        allMeal.append(j)
        

In [10]:
print(len(mealpreFeatureData[2][0]))

30


In [11]:
maxY = max(allMeal)
minY = min(allMeal)

print(minY)
print(maxY)

n = int((maxY - minY) / 20)
print(n)

40.0
400.0
18


In [12]:
transactions = []

for i in mealpreFeatureData:
    meal = []
    meal.append(max(np.array(i[0])))
    meal.append(i[0][0])
    meal.append(int(np.array(i[1])))
    transactions.append(meal)

In [13]:
#(1)40-60
#(2)60-80
#(3)80-100
#4)100-120
#(5)120-140
#(6)140-160
#(7)160-180
#(8)180-200
#(9)200-220
#10)220-240
#(11)240-260
#(12)260-280
#(13)280-300
#(14)300-320
#(15)320-340
#(16)340-360
#17)360-380
#18)380-400

In [138]:
transactionLabels = []
for i in transactions:
    arr = []
    if i[0] >=40 and i[0] <= 60:
        arr.append("Bmax1")
    if i[0] >60 and i[0] <= 80:
        arr.append("Bmax2")
    if i[0] >=80 and i[0] <= 100:
        arr.append("Bmax3")
    if i[0] >=100 and i[0] <= 120:
        arr.append("Bmax4")
    if i[0] >=120 and i[0] <= 140:
        arr.append("Bmax5")
    if i[0] >=140 and i[0] <= 160:
        arr.append("Bmax6")
    if i[0] >=160 and i[0] <= 180:
        arr.append("Bmax7")
    if i[0] >=180 and i[0] <= 200:
        arr.append("Bmax8")
    if i[0] >=200 and i[0] <= 220:
        arr.append("Bmax9")
    if i[0] >=220 and i[0] <= 240:
        arr.append("Bmax10")
    if i[0] >=240 and i[0] <= 260:
        arr.append("Bmax11")
    if i[0] >=260 and i[0] <= 280:
        arr.append("Bmax12")
    if i[0] >=280 and i[0] <= 300:
        arr.append("Bmax13")
    if i[0] >=300 and i[0] <= 320:
        arr.append("Bmax14")
    if i[0] >=320 and i[0] <= 340:
        arr.append("Bmax15")
    if i[0] >=340 and i[0] <= 360:
        arr.append("Bmax16")
    if i[0] >=360 and i[0] <= 380:
        arr.append("Bmax17")
    if i[0] >=380 and i[0] <= 400:
        arr.append("Bmax18")
        
        
        
    if i[1] >=40 and i[1] <= 60:
        arr.append("Bmeal1")
    if i[1] >60 and i[1] <= 80:
        arr.append("Bmeal2")
    if i[1] >=80 and i[1] <= 100:
        arr.append("Bmeal3")
    if i[1] >=100 and i[1] <= 120:
        arr.append("Bmeal4")
    if i[1] >=120 and i[1] <= 140:
        arr.append("Bmeal5")
    if i[1] >=140 and i[1] <= 160:
        arr.append("Bmeal6")
    if i[1] >=160 and i[1] <= 180:
        arr.append("Bmeal7")
    if i[1] >=180 and i[1] <= 200:
        arr.append("Bmeal8")
    if i[1] >=200 and i[1] <= 220:
        arr.append("Bmeal9")
    if i[1] >=220 and i[1] <= 240:
        arr.append("Bmeal0")
    if i[1] >=240 and i[1] <= 260:
        arr.append("Bmeal11")
    if i[1] >=260 and i[1] <= 280:
        arr.append("Bmeal12")
    if i[1] >=280 and i[1] <= 300:
        arr.append("Bmeal13")
    if i[1] >=300 and i[1] <= 320:
        arr.append("Bmeal14")
    if i[1] >=320 and i[1] <= 340:
        arr.append("Bmeal15")
    if i[1] >=340 and i[1] <= 360:
        arr.append("Bmeal16")
    if i[1] >=360 and i[1] <= 380:
        arr.append("Bmeal17")
    if i[1] >=380 and i[1] <= 400:
        arr.append("Bmeal18")
        
    arr.append(str(i[2]))
    transactionLabels.append(arr)




In [139]:
# df = pd.DataFrame(transactionLabels)

In [140]:
# df = df[[0,1,2]]
# df.head()

In [141]:
# basket = apriori(df,min_support=0.002,use_colnames=True)

In [445]:
association_rules_anomaly = apriori(transactionLabels, min_support=0.0000000000001, min_confidence=0.1, min_length=3)
association_rules_anomaly = list(association_rules_anomaly)

In [450]:
print("Anomalous events ","\n")

#b = set(a)
#list(b)
count = 0
for i in association_rules_anomaly:
    if len(i.ordered_statistics[0][0]) == 2 and len(i.ordered_statistics[0][1]) == 1:
        cond = list(i.ordered_statistics[0][0])[0]
        cond1 = list(i.ordered_statistics[0][0])[1]

        if cond[0:4] == "Bmax" and cond1[0:4] == "Bmea":
            cond2 = list(i.ordered_statistics[0][1])[0]
            
            if cond2[0:4] != "Bmax" and cond2[0:4] != "Bmea":

                confidence = i.ordered_statistics[0][2]
                
                if confidence <= 0.15:
                    count += 1
                    confidence = str(i.ordered_statistics[0][2])
                    print("{"+cond+","+cond1+"} -> "+cond2+"   "+confidence[0:4]+"%")

print(count)

Anomalous events  

{Bmax7,Bmeal3} -> 2   0.14%
{Bmax9,Bmeal7} -> 2   0.1%
2


In [405]:
# rules=association_rules(df,metric="lift",min_threshold=1)
association_rules_frequent = apriori(transactionLabels, min_length =1)
association_rules_frequent = list(association_rules_frequent)


In [427]:
print("Most frequent","\n")
count = 0
for i in association_rules_frequent:
        count += 1
        cond2 = list(i.ordered_statistics[0][1])[0]            
        support = str(i[1])
        print(cond2+"   "+support[0:4]+"%")
        
print(count)

Most frequent 

0   0.14%
1   0.16%
2   0.16%
3   0.11%
Bmax10   0.12%
Bmax12   0.10%
Bmax7   0.11%
Bmax8   0.13%
Bmax9   0.13%
Bmeal4   0.13%
Bmeal5   0.13%
Bmeal6   0.12%
12


In [428]:
association_rules = apriori(transactionLabels, min_support=0.0000000000001, min_confidence=0.4, min_length=3)
# association_rules = apriori(transactionLabels, min_support=0.0046, max_confidence=0.1, min_length=3)

association_results = list(association_rules)

In [430]:
print("Largest confidence","\n")

#b = set(a)
#list(b)
count = 0
for i in association_results:
    if len(i.ordered_statistics[0][0]) == 2 and len(i.ordered_statistics[0][1]) == 1:
        cond = list(i.ordered_statistics[0][0])[0]
        cond1 = list(i.ordered_statistics[0][0])[1]

        if cond[0:4] == "Bmax" and cond1[0:4] == "Bmea":
            cond2 = list(i.ordered_statistics[0][1])[0]
            
            if cond2[0:4] != "Bmax" and cond2[0:4] != "Bmea":

                count += 1
                confidence = str(i.ordered_statistics[0][2])
                print("{"+cond+","+cond1+"} -> "+cond2+"   "+confidence[0:4]+"%")

print(count)

Largest confidence 

{Bmax13,Bmeal11} -> 0   0.5%
{Bmax9,Bmeal7} -> 0   0.4%
{Bmax9,Bmeal8} -> 0   0.57%
{Bmax11,Bmeal7} -> 1   0.5%
{Bmax7,Bmeal3} -> 1   0.42%
{Bmax9,Bmeal2} -> 1   0.5%
{Bmax13,Bmeal0} -> 2   0.5%
{Bmax15,Bmeal6} -> 2   0.5%
{Bmax16,Bmeal14} -> 2   0.5%
{Bmax17,Bmeal7} -> 2   1.0%
{Bmax5,Bmeal2} -> 2   0.5%
{Bmax11,Bmeal7} -> 3   0.5%
{Bmax13,Bmeal7} -> 5   0.4%
{Bmax15,Bmeal6} -> 5   0.5%
{Bmax13,Bmeal4} -> 7   0.5%
15


In [145]:
# rules = association_results.sort_values(['confidence','lift'],ascending = [False,False])

In [146]:
# import csv

In [147]:
# df = pd.read_csv("temp.csv")

In [148]:
# with open('Results.csv','w') as fd:

#     fieldnames = ['A','B','C','D','E','F']

#     writer = csv.DictWriter(fd, fieldnames=fieldnames,lineterminator='\n')

#     writer.writerow({'A':k_meansSSE,'B':100000 ,'C':k_meansEntropy,'D':dbscanEntropy,'E':k_meansPurity,'F':dbscanPurity})
